# Atrapar porcentajes quiniela

In [ ]:
import bs4 as bs
import urllib
import urllib.request
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint
import time
import string
import pprint

import io
from urllib.request import urlopen, Request
import requests

import http.client
from http.client import IncompleteRead

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.common.exceptions import NoSuchElementException

# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

print (webdriver.__version__)

In [ ]:
URL = "https://www.eduardolosilla.es/quiniela/ayudas/"

options = webdriver.ChromeOptions()
#service = ChromeService(executable_path='chromedriver.exe')
service = ChromeService(ChromeDriverManager().install())
driverPorcentajes = webdriver.Chrome(service=service, options=options)
driverEscrutinio = webdriver.Chrome(service=service, options=options)
#print(driver)

driverPorcentajes.get(URL + "porcentajes")
driverEscrutinio.get(URL + "escrutinio")


TEMPORADA_ = 3 # 0 -> La ultima temporada.
NUM_TOTAL_JORNADAS = 200 # The algorithm already finish when no more jornadas.
MAIN_BOX_INCREMENT_BETWEEN_MATCHES = 5
NUM_MATCHES_IN_A_JORNADA = 14
PLENO_15_ADDITIONAL_ROWS = 2

dfTotal = pd.DataFrame()
columns = columns=['Season', 'Jornada', 'Line', 'LocalTeamName', 'AwayTeamName', 'Result',
                   'PorcentajeQuinielistaLocal', 'PorcentajeQuinielistaDraw', 'PorcentajeQuinielistaAway',
                   'PorcentajeLaeLocal',         'PorcentajeLaeDraw',         'PorcentajeLaeAway',
                   'PorcentajeRealLocal',        'PorcentajeRealDraw',        'PorcentajeRealAway',
                   'Acertantes15', 'Premio15', 'Acertantes14', 'Premio14', 'Acertantes13', 'Premio13',
                   'Acertantes12', 'Premio12', 'Acertantes11', 'Premio11', 'Acertantes10', 'Premio10',
                   'AcertantesElige8', 'PremioElige8'
                  ]

jornadaList = []
jornada = -1

select = Select(driverPorcentajes.find_element("xpath","//*[starts-with(@class,'c-selector u-custom-select temporada')]/select"))
select.select_by_value( str(TEMPORADA_) )
time.sleep(1)
selectEscr = Select(driverEscrutinio.find_element("xpath","//*[starts-with(@class,'c-selector u-custom-select temporada')]/select"))
selectEscr.select_by_value(str(TEMPORADA_) )
time.sleep(2)

# for every "jornada"
while jornada < NUM_TOTAL_JORNADAS:
    jornada += 1
    
    time.sleep(randint(2,3))  
    
    #####################################
    # Click on the "jornada" tab
    #####################################


    try:
        select = Select(driverPorcentajes.find_element("id",'u-hidden-scrollbar'))
        select.select_by_value(str(jornada))
        
        selectEscr = Select(driverEscrutinio.find_element("id",'u-hidden-scrollbar'))
        selectEscr.select_by_value(str(jornada))
        
        time.sleep(2)
        
        # Selecting Temporada again because Escrutinio web has a bug that the temporada tab goes mad when changing the jornada.
        selectEscr = Select(driverEscrutinio.find_element("xpath","//*[starts-with(@class,'c-selector u-custom-select temporada')]/select"))
        selectEscr.select_by_value(str(TEMPORADA_) )
        
    except NoSuchElementException:
        jornada = NUM_TOTAL_JORNADAS
    
    time.sleep(randint(2,4))  
    
    
    #####################################
    # Find main Box
    #####################################    
    mainBoxes = driverPorcentajes.find_elements("xpath","//div[@class='c-caja_base m-no-fixed-size']")
    
    results = driverPorcentajes.find_elements("xpath","//button[@class='u-pronostico m-signo']")
    #print(results[0].text)
    
    for mainBox in mainBoxes:
        #print(mainBox.text)
        mainBoxTable = mainBox.text.split('\n')
        #print(cajaTable)  # ['1', 'R.SOCIEDAD - GIRONA', '1', 'X', '2', '2', 'LAS PALMAS - MALLORCA', '1', 'X', '2', '3', 'ATH.CLUB - R.MADRID', '1', 'X', '2', . . .

        jornadaBox = driverPorcentajes.find_elements("xpath","//span[@class='c-selector-horizontal__content__text']")
        #print(jornadaBox[0].text)
        saux = (jornadaBox[0].text).split(' - ')
        jornada_ = [int(s) for s in saux[0].split() if s.isdigit()][0]
        season_ = saux[1]
        
        porcentajesTable = driverPorcentajes.find_elements("xpath","//div[@class='c-porcentajes__fila__container u-clearfix ng-star-inserted']")
        
        #####################################
        ### Escrutinio tab
        #####################################
        cajasRecaudacion = driverEscrutinio.find_elements("xpath","//table[@class='c-tabla-categorias__table']")
        recaudacionList = []
        for result in cajasRecaudacion:
            recaudacionTable = result.text.split(' ')
            recaudacionList.append(recaudacionTable)
        #print(recaudacionList)
        
        acertantesAciertos15_ = recaudacionList[0][3]
        premioAciertos15_     = int("".join(filter(str.isdigit, recaudacionList[0][4][0:(recaudacionList[0][4]).find(',')])))
        acertantesAciertos14_ = recaudacionList[0][6]
        premioAciertos14_     = int("".join(filter(str.isdigit, recaudacionList[0][7][0:(recaudacionList[0][7]).find(',')])))
        acertantesAciertos13_ = recaudacionList[0][9]
        premioAciertos13_     = int("".join(filter(str.isdigit, recaudacionList[0][10][0:(recaudacionList[0][10]).find(',')])))
        acertantesAciertos12_ = recaudacionList[0][12]
        premioAciertos12_     = int("".join(filter(str.isdigit, recaudacionList[0][13][0:(recaudacionList[0][13]).find(',')])))
        acertantesAciertos11_ = recaudacionList[0][15]
        premioAciertos11_     = int("".join(filter(str.isdigit, recaudacionList[0][16][0:(recaudacionList[0][16]).find(',')])))
        acertantesAciertos10_ = recaudacionList[0][18]
        premioAciertos10_     = int("".join(filter(str.isdigit, recaudacionList[0][19][0:(recaudacionList[0][19]).find(',')])))
        if(len(recaudacionList[0]) > 25):
            acertantesElige8_     = recaudacionList[0][27]
            premioElige8_         = int("".join(filter(str.isdigit, recaudacionList[0][28][0:(recaudacionList[0][28]).find(',')])))
        else:
            acertantesElige8_ = -1
            premioElige8_ = -1
    

        ii_mainBoxTable = 0 # be careful not incrementing ++ (by one)
        while(ii_mainBoxTable < (MAIN_BOX_INCREMENT_BETWEEN_MATCHES * NUM_MATCHES_IN_A_JORNADA)):
            
            #####################################
            ### Main and porcentajes tab
            #####################################
            line_ = mainBoxTable[ii_mainBoxTable]
            #print(line_)
            saux = (mainBoxTable[ii_mainBoxTable+1]).split(' - ')
            localTeamName_ = saux[0]
            awayTeamName_ = saux[1]
            #print(localTeamName_)
            #print(awayTeamName_)

            saux = porcentajesTable[int(line_)-1].text.split('\n')
            porcentajeQuinielistaLocal_ = saux[0]
            porcentajeQuinielistaDraw_  = saux[1]
            porcentajeQuinielistaAway_  = saux[2]
            
            saux = porcentajesTable[int(line_)-1 + (NUM_MATCHES_IN_A_JORNADA+PLENO_15_ADDITIONAL_ROWS)*1].text.split('\n')
            porcentajeLaeLocal_ = saux[0]
            porcentajeLaeDraw_  = saux[1]
            porcentajeLaeAway_  = saux[2]
            
            saux = porcentajesTable[int(line_)-1 + (NUM_MATCHES_IN_A_JORNADA+PLENO_15_ADDITIONAL_ROWS)*2].text.split('\n')
            porcentajeRealLocal_ = saux[0]
            porcentajeRealDraw_  = saux[1]
            porcentajeRealAway_  = saux[2]

            row = [season_, jornada_, line_, localTeamName_, awayTeamName_, 
                   results[int(ii_mainBoxTable/MAIN_BOX_INCREMENT_BETWEEN_MATCHES)].text,
                   porcentajeQuinielistaLocal_, porcentajeQuinielistaDraw_, porcentajeQuinielistaAway_,
                   porcentajeLaeLocal_,         porcentajeLaeDraw_,         porcentajeLaeAway_,
                   porcentajeRealLocal_,        porcentajeRealDraw_,        porcentajeRealAway_,
                   acertantesAciertos15_, premioAciertos15_, acertantesAciertos14_, premioAciertos14_,                         
                   acertantesAciertos13_, premioAciertos13_, acertantesAciertos12_, premioAciertos12_,
                   acertantesAciertos11_, premioAciertos11_, acertantesAciertos10_, premioAciertos10_,
                   acertantesElige8_, premioElige8_  
                  ]
            
            jornadaList.append(row)
  
            ii_mainBoxTable += MAIN_BOX_INCREMENT_BETWEEN_MATCHES

        #print(jornadaList)

df = pd.DataFrame(jornadaList, columns=columns)
df

In [ ]:
df

In [ ]:
df.to_csv("Quiniela_2324Partial.csv", index=False, header=True)

-----